In [1]:
import jams
import os
import matplotlib.pyplot as plt
#este modulo es local (interpreter.py) y contiene las funciones que se usan para visualizar los jams
import interpreter as itp
## Change This Cell!
gs_path = "./data/"
anno_dir = "annotation/"
excerpt = "02_Rock1-90-C#_solo"

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
jam = jams.load(os.path.join(gs_path, anno_dir, excerpt + ".jams"))

In [9]:
jam.search(namespace='tempo')

,time,duration,value,confidence
0,0.000,32.000,90.0,1.0


In [8]:
jam.search(namespace='note_midi')[1].data

SortedKeyList([Observation(time=4.662967800453515, duration=0.33088435374149583, value=51.13379004677491, confidence=None), Observation(time=5.0164145124716555, duration=0.38312925170068013, value=49.13097682764926, confidence=None), Observation(time=9.02872743764172, duration=0.16253968253968054, value=51.17600938142521, confidence=None), Observation(time=9.191924716553288, duration=0.13351473922902812, value=51.11981245254794, confidence=None), Observation(time=9.34278639455782, duration=0.33088435374149583, value=51.15813989268465, confidence=None), Observation(time=15.014419047619043, duration=0.3018594104308363, value=51.13300405733549, confidence=None), Observation(time=15.338387301587304, duration=0.3018594104308363, value=49.184199600871615, confidence=None), Observation(time=15.641607256235822, duration=0.3366893424036306, value=46.18765526174031, confidence=None), Observation(time=15.999747845804983, duration=0.38312925170068013, value=49.22167312882971, confidence=None), Obs

In [4]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import os
import jams
import mido

def write_midi(inp_file, output_file=None):
    #si ya es .midi, no haga nada
    _, ext = os.path.splitext(inp_file)
    if ext == '.midi':
        return
    jam = jams.load(inp_file) 
    notes = jam.search(namespace='note_midi')
    bpm = jam.search(namespace='tempo')[0].data[0].value
    all_notes = [list(note.data) for note in notes]
    strings = [[int(note.annotation_metadata.data_source)+1]*len(note.data) for note in notes]
    strings = [string for sublist in strings for string in sublist]
    all_notes = [note for sublist in all_notes for note in sublist]
    all_notes_with_strings = list(zip(all_notes, strings))
    strings = sorted(all_notes_with_strings, key=lambda x: x[0].time)
    mid = MidiFile()
    tempo = mido.bpm2tempo(bpm)
    ticks_per_beat = mid.ticks_per_beat
    tracks = []
    for string in range(6):
        track = MidiTrack()
        mid.tracks.append(track)
        # Set tempo
        track.append(MetaMessage('set_tempo', tempo=tempo, time=0))
        # Set instrument to Guitar (program number 26)
        track.append(Message('program_change', program=26, time=0))
        track.append(MetaMessage('time_signature', numerator=4, denominator=4, time=0))
        tracks.append(track)

    last_tick = [0] * 6  # tick de cada cuerda

    for note in strings:
        duration = note[0].duration
        start_time = note[0].time
        pitch_freq = note[0].value
        midi_note = int(pitch_freq)
        
        start_tick = mido.second2tick(start_time, ticks_per_beat, tempo)
        duration_ticks = mido.second2tick(duration, ticks_per_beat, tempo)
        
        string = note[1] - 1
        delta_time = max(0, int(start_tick - last_tick[string]))  # Ensure non-negative delta_time

        #time not absolute, relative u.u 
        tracks[string].append(Message('note_on', note=midi_note, velocity=64, time=delta_time))
        tracks[string].append(Message('note_off', note=midi_note, velocity=64, time=duration_ticks))
        
        last_tick[string] = start_tick + duration_ticks #el ultimo evento fue en este tick
    if output_file:
        mid.save(output_file)
    else:
        mid.save(inp_file.replace('.jams', '.mid'))





In [5]:
write_midi('./data/annotation/02_Rock1-90-C#_solo.jams','./data/transformed/02_Rock1-90-C#_solo.mid')

In [ ]:
for raiz, dirs, archivos in os.walk(directorio):
    for archivo in archivos:
        ruta_completa = os.path.join(raiz, archivo)
        print(ruta_completa)
        try:
            write_midi(ruta_completa)
        except Exception as e:
            print(f"Error al convertir el archivo {ruta_completa}: {e}")
            continue

In [7]:
import music21
#this one works
# Specify the path to your MIDI file
midi_path = "./data/transformed/02_Rock1-90-C#_solo.mid"
mf = music21.midi.MidiFile()
mf.open(midi_path)
mf.read()
mf.close()

# Convert MIDI file to music21 stream
midi_stream = music21.midi.translate.midiFileToStream(mf)
jam = jams.load('./data/annotation/02_Rock1-90-C#_solo.jams')
# Assuming the existence of some `jam` object or data that you fetch elsewhere
# For this example, let's create some dummy chord data to work with
# Example chord data and tempo - you should replace this with your actual data
chords = jam.search(namespace='chord')[0].data
tempo = jam.search(namespace='tempo')[0].data[0].value # bpm, this should come from your `jam` object

# You might want to process the first part only if there are multiple parts
part = midi_stream.getElementsByClass('Part')
max_part = max(part, key=lambda part: len(part.getElementsByClass('Measure')))
# Process each chord
for chord_data in chords:
    chord_name = chord_data.value.replace(':', '').replace('hdim7', 'm7b5')
    print(chord_name)
    chord_duration = chord_data.duration
    # Calculate the duration in quarter lengths assuming 60 seconds per minute
    chord_duration_quarters = (tempo / 60) * chord_duration
    # Calculate where to insert the chord in terms of measures
    start_measure = int((chord_data.time * tempo / 60) / 4)  # Simplified estimate
    print(start_measure)
    chord_obj = music21.harmony.ChordSymbol(chord_name)
    chord_obj.duration.quarterLength = chord_duration_quarters
    max_part.getElementsByClass('Measure')[start_measure].insert(0, chord_obj)  # measures are 1-indexed

# Set metadata
midi_stream.metadata = music21.metadata.Metadata()
midi_stream.metadata.title = "02_Rock1-90-C#_solo"

# Save as MusicXML (for viewing in notation software)
midi_stream.write('musicxml', '02_Rock1-90-C#_solo.xml')




C#maj
0
F#maj
4
C#maj
6
G#maj
8
F#maj
9
C#maj
10


WindowsPath('c:/Users/acmor/Documents/code/IA/ML_course-2024/proyecto/02_Rock1-90-C#_solo.xml')